In [546]:
from datetime import datetime, date, timedelta
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass, field
import types
from pprint import pprint

class Holiday:
    def __init__(self,name, date):
        self.name = name
        # only accept datetime objects as date
        self.date = datetime.strptime(date, "%Y-%m-%d")
    
    def __str__(self):
        return f"{self.name} ({self.date})"

In [547]:
#Need to uncomment print in addHoliday function

class HolidayList:
    "HolidayList class"
    def __init__(self):
        self.innerHolidays = []

    def addHoliday(self, holidayObj):
        if isinstance(holidayObj, Holiday):
            # Make sure holidayObj is an Holiday Object by checking the type. Add to innerHolidays
            self.innerHolidays.append(holidayObj)
            #print(f"Added holiday {holidayObj}")

    def findHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            # Find and return Holiday in innerHolidays
            if holiday.name == HolidayName and holiday.date == Date:
                return holiday
            else:
                print(f"{holiday} is not in list.")

    def removeHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            # Find Holiday in innerHolidays by searching the name and date combination.
            if holiday.name == HolidayName and holiday.date == Date:
                # remove the Holiday from innerHolidays, and inform user you deleted the holiday
                self.innerHolidays.remove(holiday)
                print(f"{holiday} was removed from the list.")
            else:
                print(f"{holiday} is not in list.")

    def read_json(self, filelocation):
        # Read in things from json file location and add holidays to innerHolidays
        with open(filelocation, "r") as jsonfile:
            data = json.load(jsonfile)
            for i in range(len(data["holidays"])):
                holiday = Holiday(data["holidays"][i]["name"], data["holidays"][i]["date"])
                HolidayList.addHoliday(self, holiday)

    def save_to_json(self, filelocation):
        json_list = []
        json_dict = {"holidays": json_list}
        # Write out json file to selected file.
        with open(filelocation, "w") as jsonfile:
            for holiday in self.innerHolidays:
                json_list.append(holiday.__dict__)
            json.dump(json_dict, jsonfile, indent=3)

    def scrapeHolidays(self):
        def getHTML(url):
            response = requests.get(url)
            return response.text
        # Scrape holidays from 2020-2024
        for year in range(2020,2025):
            year = str(year)
            html = getHTML("https://www.timeanddate.com/calendar/custom.html?year=" + year +"&country=1&cols=3&df=1&hol=33554809")
            soup = BeautifulSoup(html,'html.parser')
            holiday_data = soup.find('table', attrs = {'class':'cht lpad'}).find('tbody')
            for row in holiday_data:
                items = row.find_all('tr')
                holiday_dict = {}
                holiday_dict['name'] = row.find('td').find_next('td').string
                holiday_dict['date'] = f"{row.find('td').string}, {year}"
                # Check to see if name and date of holiday is in innerHolidays array. If it is, don't add it to list
                if holiday_dict['name'] in self.innerHolidays and holiday_dict['date'] in self.innerHolidays:
                    continue
                else:
                    # converting date to correct format
                    holiday_dict['date'] = str(datetime.strptime(holiday_dict['date'], "%b %d, %Y"))[0:10]
                    # add holiday to list
                    holiday = Holiday(holiday_dict["name"], holiday_dict["date"])
                    HolidayList.addHoliday(self, holiday)
    
    def numHolidays(self):
        # Return the total number of holidays in innerHolidays
        return len(self.innerHolidays)

    def filter_holidays_by_week(self, year, week_number):
        # Use a Lambda function to filter by week number and cast results as list
        global filter_by_week
        filter_by_week = filter(lambda holiday:datetime.strftime(holiday.date, "%U") == week_number
            and datetime.strftime(holiday.date, "%Y") == year, self.innerHolidays)
        filter_by_week = list(filter_by_week)
        return(filter_by_week)
    
    def displayHolidaysInWeek(self, year, week_number):
        # Print list of holidays within a week as a parameter
        HolidayList.filter_holidays_by_week(self, year, week_number)
        for row in range(len(filter_by_week)):
            print(filter_by_week[row])
    
    def getWeather_currentweek(self):
        # Convert weekNum to range between two days (Is this necessary?)
        #year_week = f"{year}-{week_number}"
        #weekNum = []
        #for i in range(0,7):
        #    day_week = datetime.strptime(f"{year_week}-{i}", "%Y-%U-%w")
        #    day_week = str(day_week)
        #    weekNum.append(day_week)
        #print(weekNum)
        # Query API for weather in current week
        url = "https://community-open-weather-map.p.rapidapi.com/forecast/daily"
        querystring = {"q":"new york,us","lat":"41","lon":"74","cnt":"7","units":"imperial"}
        headers = {'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
            'x-rapidapi-key': "1e19830f3bmshf56abb61a6e9168p189818jsnf035a12afc26"}
        weather = requests.request("GET", url, headers=headers, params=querystring)
        weather_dict = weather.json()
        current_week_weather = []
        # return weather string for current week
        for i in range(0,7):
            daily_weather = (weather_dict["list"][i]["weather"][0]["main"])
            current_week_weather.append(daily_weather)
        return(current_week_weather)
    
    def viewCurrentWeek(self):
        # Use the Datetime Module to look up current week and year
        today = date.today()
        this_week = []
        for i in range(0,7):
            this_week.append(str(today + timedelta(days=i)))
        print(this_week)
        current_year = datetime.strftime(today, "%Y")
        current_week = datetime.strftime(today, "%U")
        # Use your filter_holidays_by_week function to get the list of holidays for the current week and year
        HolidayList.filter_holidays_by_week(self, current_year, current_week)
        # Use your displayHolidaysInWeek function to display the holidays in the week
        HolidayList.displayHolidaysInWeek(self, current_year, current_week)
        # Ask user if they want to get the weather
        weather_yn  = str(input("Would you like to see this week's weather? [y/n]"))
        while True:
            weather_yn  = str(input("Would you like to see this week's weather? [y/n]"))
            # If yes, use your getWeather function and display results
            if weather_yn == "y":
                current_week_weather = HolidayList.getWeather_currentweek(self)
                current_weather_date = {}
                for i in range(0,7):
                    current_weather_date['date'] = this_week[i]
                    current_weather_date['weather'] = current_week_weather[i]
                    print(current_weather_date)
                break
            elif weather_yn == "n":
                break
            else:
                print("I didn't understand. Try again.")

In [548]:
holiday1 = Holiday("New Year's Day", "2022-01-01")
print(holiday1)

holidaylist = HolidayList()

holidaylist.addHoliday(holiday1)
#holidaylist.findHoliday("New Year's Day", "2022-01-01")
#holidaylist.removeHoliday("New Year's Day", "2022-01-01")

holidaylist.read_json("holidays.json")

holidaylist.scrapeHolidays()

#holidaylist.numHolidays()

#holidaylist.displayHolidaysInWeek("2022", "00")
#holidaylist.getWeather_currentweek()

holidaylist.viewCurrentWeek()




New Year's Day (2022-01-01 00:00:00)
['2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15']
Stephen Foster Memorial Day (2022-01-13 00:00:00)
{'date': '2022-01-09', 'weather': 'Rain'}
{'date': '2022-01-10', 'weather': 'Clouds'}
{'date': '2022-01-11', 'weather': 'Snow'}
{'date': '2022-01-12', 'weather': 'Clouds'}
{'date': '2022-01-13', 'weather': 'Clouds'}
{'date': '2022-01-14', 'weather': 'Clouds'}
{'date': '2022-01-15', 'weather': 'Snow'}
